<br><br>
<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC', 'Inconsolata'; font-size: 440%; font-weight: 700; text-align: center; color: DarkSeaGreen;">
實作一個自訂的set
</div>
<br><br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 實作完整的set談何容易，要顧及的底層細節非常多，很可能超出筆者的能力。
* 所以本文的自訂set，只是修改Python set的一小部分，練功成份大於實用。

</div>
<br><br>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
先複習Python的set
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* set的特性
* set的notation
* 建立空set
* 運算子和方法

<div style="color: tomato; text-align: center;">
(各位請打開麥克風)
</div>

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
set的特性
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 每個元素都是唯一(unique)不會重複。
* 元素的順序無法保證(unordered)。 
* 可新增、刪除元素，但不能修改元素的值。

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
set的notation
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* set是用大括號(花括弧)`{}`作為前後綴符號，例如：
* `set1 = {'Python', 'JS', 'C++', 'Java'}`

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
建立空set
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 150%; color: Gainsboro">

1. `set1 = set()`
2. `set2 = {}`

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
運算子和方法
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* set提供以下運算子(operators)和對應方法(methods)：
    <div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 70%; color: Gainsboro">
    
    |運算子|方法|中文|範例 / 說明|
    |--|--|--|--|
    |\||union()|聯集|<ol><li>set1 \| set2</li><li>set1.union(set2)</li></ol>|
    |&|intersection()|交集|<ol><li>set1 & set2</li><li>set1.intersection(set2)|
    |-|difference()|差集|<ol><li>set1 - set2</li><li>set1.difference(set2)|
    |^|symmetric_difference()|對稱差|<ol><li>set1 ^ set2</li><li>set1.symmetric_difference(set2)|
    |<=|issubset()|子集|<ol><li>set1 <= set2</li><li>set1.issubset(set2)|
    |<|issubset() and ...|真子集|<ol><li>set1 < set2</li><li>set1.issubset(set2) and<br>set1 != set2|
    |>=|issuperset()|超集|<ol><li>set1 >= set2</li><li>set1.issuperset(set2)|
    |>|issuperset() and ...|真超集|<ol><li>set1 > set2</li><li>set1.issuperset(set2) and<br>set1 != set2|
    |\|=|update()||add elements<br>from all others.|
    |&=|intersection_update()||keep only elements<br>found in it<br>and all others.|
    |-=|difference_update()||remove elements<br>found in others.|
    |^=|symmetric_difference_update()||keep only elements<br>found in either set,<br>but not in both.|
    </div>
* Python的set沒有提供`+`和`*`運算子。
</div>

In [22]:
set1 = {1, 2, 3, 4, 5}
set2 = {8, 7, 6, 5, 4}

In [26]:
# union
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 | set2      = }')
print(f'{set1.union(set2) = }')

set1 = {1, 2, 3, 4, 5}
set2 = {4, 5, 6, 7, 8}
set1 | set2      = {1, 2, 3, 4, 5, 6, 7, 8}
set1.union(set2) = {1, 2, 3, 4, 5, 6, 7, 8}


In [28]:
# intersection
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 & set2             = }')
print(f'{set1.intersection(set2) = }')

set1 = {1, 2, 3, 4, 5}
set2 = {4, 5, 6, 7, 8}
set1 & set2             = {4, 5}
set1.intersection(set2) = {4, 5}


In [29]:
# difference
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 - set2           = }')
print(f'{set1.difference(set2) = }')

set1 = {1, 2, 3, 4, 5}
set2 = {4, 5, 6, 7, 8}
set1 - set2           = {1, 2, 3}
set1.difference(set2) = {1, 2, 3}


In [31]:
# symmetric difference
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 ^ set2                     = }')
print(f'{set1.symmetric_difference(set2) = }')

set1 = {1, 2, 3, 4, 5}
set2 = {4, 5, 6, 7, 8}
set1 ^ set2                     = {1, 2, 3, 6, 7, 8}
set1.symmetric_difference(set2) = {1, 2, 3, 6, 7, 8}


In [36]:
# subset
set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 <= set2        = }')
print(f'{set1.issubset(set2) = }')

set1 = {1, 2, 3, 4, 5}
set2 = {3, 4, 5}
set1 <= set2        = False
set1.issubset(set2) = False


In [35]:
# superset
set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 >= set2          = }')
print(f'{set1.issuperset(set2) = }')

set1 = {1, 2, 3, 4, 5}
set2 = {3, 4, 5}
set1 >= set2          = True
set1.issuperset(set2) = True


<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
set的其他方法
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 除了以上的運算外，set還有好幾個方法可用：
    <div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 90%; color: Gainsboro">
    
    |方法|說明|範例|
    |--|--|--|
    |add()|新增元素||
    |remove()|刪除元素||
    |discard()|刪除元素||
    |pop()|||
    |clear()|||
    |isdisjoint()|||
    |copy()|shallow copy||
    </div>
* 一開始講set特性時有提到不能修改元素的值，所以沒有replace()方法。

</div>

In [38]:
# 3 46 40
3*3600 + 46*60 + 40
3*3600 + 48*60 + 55


13600

13735

<div style="text-align:center"><img src="https://hackmd.io/_uploads/S1ZVMQOP6.png" width="500"/></div>

<div style="text-align:center"><img src="https://hackmd.io/_uploads/ryRk_EdwT.jpg" width="300"/></div>

In [20]:
# 建立空set。
set1 = set()
set2 = {}

print(f'{set1 = }\t{type(set1) = }')
print(f'{set2 = }\t{type(set2) = }')

set1 = set()	type(set1) = <class 'set'>
set2 = {}	type(set2) = <class 'dict'>


In [7]:
from typeutil import is_hashable


class RobustSet(set):
    def __init__(self, s: set):
        super().__init__()
        self.__set = s

    def add(self, elements: list | tuple | set):
        # if set(hashable_elements := [element for element in elements if is_hashable(element)]).issubset(self.__set):
        tmp_set = self.__set.copy()
        # added_count = 0
        for element in (hashable_elements := [element for element in elements if is_hashable(element)]):
            self.__set.add(element)

        return len(self.__set) - len(tmp_set)

    def discard(self, element: set):
        if element in self.__set:
            return_value = True
        else:
            return_value = False
        self.__set.discard(element)
        return return_value

    def remove(self, elements: list | tuple | set):
        if set(hashable_elements := [element for element in elements if is_hashable(element)]).issubset(self.__set):
            return_value = True
        else:
            return_value = False

        for hashable_element in hashable_elements:
            if hashable_element in self.__set:
                self.__set.remove(hashable_element)
        return return_value

    def replace(self, old: list | tuple | set, new: object) -> None:
        self.remove(old)
        self.add(new)
        return None


    def show(self):
        return self.__set

# my_set = RobustSet({4, 7, 3, 3, 2, 0, 6})
# my_set = RobustSet({1, 8, 3, 0})
# my_set.show()

# print(my_set.discard(45))
# my_set.show()

# print(my_set.remove([45, 4, [7], 1, 'A', False, True, None]))
# my_set.show()
# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.remove([4, 8])
# my_set.show()

# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.remove((4, 8))
# my_set.show()

# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.remove({4, 8})
# my_set.show()


my_set = RobustSet({1, 8, 7, 0, 9, 3})
my_set.show()
my_set.add([4, [8], -0, {1}, '9', 4, 3, 2, 3])
my_set.show()

{0, 1, 3, 7, 8, 9}

3

{0, 1, 2, 3, 4, 7, 8, 9, '9'}

In [4]:
s = {6, 8, 3, 5}
s
print(s.add(8))
s

{3, 5, 6, 8}

None


{3, 5, 6, 8}